In [1]:
import os
import ctypes
from pathlib import Path

# Use absolute path to acados (same as run.sh)
ACADOS_PATH = "/root/robotic-mpc/acados"
ACADOS_LIB_PATH = f"{ACADOS_PATH}/lib"

os.environ["ACADOS_SOURCE_DIR"] = ACADOS_PATH
os.environ["LD_LIBRARY_PATH"] = f"{ACADOS_LIB_PATH}:" + os.environ.get("LD_LIBRARY_PATH", "")

# Preload shared libraries (LD_LIBRARY_PATH set at runtime doesn't work for dynamic linker)
# Load in dependency order
for lib in ["libblasfeo.so", "libhpipm.so", "libqpOASES_e.so", "libacados.so"]:
    lib_path = f"{ACADOS_LIB_PATH}/{lib}"
    try:
        ctypes.CDLL(lib_path, mode=ctypes.RTLD_GLOBAL)
        print(f"✓ Loaded {lib}")
    except OSError as e:
        print(f"✗ Failed to load {lib}: {e}")

print(f"\nACADOS_SOURCE_DIR = {os.environ['ACADOS_SOURCE_DIR']}")

# Verify t_renderer exists
tera_path = Path(ACADOS_PATH) / "bin" / "t_renderer"
print(f"t_renderer exists: {tera_path.exists()} at {tera_path}")


✓ Loaded libblasfeo.so
✓ Loaded libhpipm.so
✓ Loaded libqpOASES_e.so
✓ Loaded libacados.so

ACADOS_SOURCE_DIR = /root/robotic-mpc/acados
t_renderer exists: True at /root/robotic-mpc/acados/bin/t_renderer


In [2]:
import sys
from pathlib import Path
from simulator import Simulator, SimulationManager
import numpy as np

BASE_SURFACE_CONFIG = np.array({'a': -0.1, 'b': 0.1, 'c': -0.01, 'd': 0.01, 'e': 0.01, 'f': 0.0})
CROSS_OVER_FREQUENCIES = np.array([228.9, 262.09, 517.3, 747.44, 429.9, 1547.76]) # Simone's Magic Formula
SAMPLING_TIME = np.pi / (5*np.max(CROSS_OVER_FREQUENCIES)) # Shannon's theorem
JOINT_POSITION_LIMITS = np.array([+2*np.pi, +2*np.pi, +2*np.pi, +2*np.pi, +2*np.pi, +2*np.pi], dtype=float)
JOINT_VELOCITY_LIMITS_UP = np.array([2.16, 2.16, np.pi, 3.20, 3.20, 3.20], dtype=float)
JOINT_VELOCITY_LIMITS_LOW = np.array([-2.16, -2.16, -np.pi, -3.20, -3.20, -3.20], dtype=float)

base_config = {
    'robot_name': 'ur10',
    'dt': SAMPLING_TIME,
    'simulation_time': 3.0,
    'prediction_horizon': 100,
    'surface_limits': ((-2, 2), (-2, 2)),
    'surface_origin': np.array([0.0, 0.0, 0.0]),
    'surface_orientation_rpy': np.array([0.0, 0.0, 0.0]),
    'q_0': np.array([np.pi/3, -np.pi/3, np.pi/4, -np.pi/2, -np.pi/2, 0.0]),
    'qdot_0': np.array([2, 0, 0, -1, 1, 1]),
    'wcv': np.array([228.9, 262.09, 517.3, 747.44, 429.9, 1547.76], dtype=float),
    'q_min': np.array([-2*np.pi, -2*np.pi, -2*np.pi, -2*np.pi, -2*np.pi, -2*np.pi], dtype=float),
    'q_max': np.array([+2*np.pi, +2*np.pi, +2*np.pi, +2*np.pi, +2*np.pi, +2*np.pi], dtype=float),
    'qdot_min': np.array([-2.16, -2.16, -np.pi, -3.20, -3.20, -3.20], dtype=float),
    'qdot_max': np.array([2.16, 2.16, np.pi, 3.20, 3.20, 3.20], dtype=float),
    'scene': True
    }
manager = SimulationManager(base_config)

In [3]:
# Extract the dictionary from the numpy array
base_dict = BASE_SURFACE_CONFIG.item()

# Generate 3 varied surface configurations
np.random.seed(42)  # For reproducibility, remove if you want different sim_results each time
n_configs = 1
std = 0.0

surface_configs = []
for i in range(n_configs):
    config = {}
    for key, mean in base_dict.items():
        # Sample from normal distribution with mean from base config and std of 1
        config[key] = np.random.normal(mean, std)
    surface_configs.append(config)

# Print the generated configs to verify
print(f"Generated {len(surface_configs)} surface configurations:")
for i, config in enumerate(surface_configs):
    print(f"Config {i}: {config}")



Generated 1 surface configurations:
Config 0: {'a': -0.1, 'b': 0.1, 'c': -0.01, 'd': 0.01, 'e': 0.01, 'f': 0.0}


In [ ]:
import os

os.environ["ACADOS_SOURCE_DIR"] = "/root/robotic-mpc/acados"
os.environ["LD_LIBRARY_PATH"] = "/root/robotic-mpc/acados/lib:" + os.environ.get("LD_LIBRARY_PATH", "")

# Run grid search with the varied configurations
manager.clear()
prediction_horizons = [400]
w_u = [0.01]
manager.grid_search(

    param_grid={
        'prediction_horizon': prediction_horizons,
        'w_u':w_u
    },
    surface_coeff_sets=surface_configs,
    name_template=lambda p, c: f"surf{c}_H{p['prediction_horizon']}"
)

sim_results = manager.run_all()


[1/1] Running: surf0_H300
URDF successfully loaded: /root/robotic-mpc/ur_description/urdf/ur10.urdf
nq = 6, ngeoms(col) = 8, ngeoms(vis) = 7
Pinocchio model and data successfully created.
rm -f libacados_ocp_solver_six_dof_robot_4d33b293.so
rm -f six_dof_robot_4d33b293_cost/six_dof_robot_4d33b293_cost_y_0_fun.o six_dof_robot_4d33b293_cost/six_dof_robot_4d33b293_cost_y_0_fun_jac_ut_xt.o six_dof_robot_4d33b293_cost/six_dof_robot_4d33b293_cost_y_0_hess.o six_dof_robot_4d33b293_cost/six_dof_robot_4d33b293_cost_y_fun.o six_dof_robot_4d33b293_cost/six_dof_robot_4d33b293_cost_y_fun_jac_ut_xt.o six_dof_robot_4d33b293_cost/six_dof_robot_4d33b293_cost_y_hess.o acados_solver_six_dof_robot_4d33b293.o
cc -fPIC -std=c99   -O2 -I/root/robotic-mpc/acados/include -I/root/robotic-mpc/acados/include/acados -I/root/robotic-mpc/acados/include/blasfeo/include -I/root/robotic-mpc/acados/include/hpipm/include  -c -o six_dof_robot_4d33b293_cost/six_dof_robot_4d33b293_cost_y_0_fun.o six_dof_robot_4d33b293_cost

In [7]:
from plotter import Plotter
plotter = Plotter()

selected_sim_results_100 = [r for r in sim_results if r['name'] == 'surf0_H300']
#selected_sim_results_20  = [r for r in sim_results if r['name'] == 'surf0_H20']

r100 = selected_sim_results_100[0]
#r20  = selected_sim_results_20[0]

p_ee_y_100 = r100['analysis']['p_ee_y']      # shape: (N,)
qddot_100  = r100['data']['qddot']           # shape: (6, N)
#p_ee_y_20 = r20['analysis']['p_ee_y']      # shape: (N,)
#qddot_20  = r20['data']['qddot']           # shape: (6, N)


fig_acc_100 = plotter.generic_plot(
    p_ee_y_100,
    *[qddot_100[j, :] for j in range(qddot_100.shape[0])],
    xlabel="Horizontal position $[m]$",
    ylabel="Joint acceleration $[rad/s^2]$",
    title="$N=100$",
    labels=[]
)
'''
fig_acc_20 = plotter.generic_plot(
    p_ee_y_20,
    *[qddot_20[j, :] for j in range(qddot_20.shape[0])],
    xlabel="Horizontal position $[m]$",
    ylabel="Joint acceleration $[rad/s^2]$",
    title="$N=20$",
    labels=[]
)'''

plotter.show(fig_acc_100)
#plotter.show(fig_acc_20)


/root/miniforge3/lib/python3.12/site-packages/plotly/offline/offline.py:152: UserWarning:


Unrecognized config options supplied: ['mathjax']



In [9]:
from plotter import Plotter
plotter = Plotter()

# Select specific simulations for detailed plots
selected_sim_results = [r for r in sim_results if r['name'] in ['surf0_H300']]
# Get time vector from first selected result
t = selected_sim_results[0]['data']['time']
i = np.arange(selected_sim_results[0]['simulator'].Nsim)

box_data = {}
for r in sim_results:
    group_label = str(r['simulator'].prediction_horizon)
    if group_label not in box_data:
        box_data[group_label] = []
    box_data[group_label].append(r['summary']['rmse_e1'])

fig_box = plotter.box_plot(
    data=box_data,
    xlabel="Prediction Horizon",
    ylabel="RMSE e1 [m]",
    title="Surface Tracking Error Distribution vs Prediction Horizon"
)

# Error tracking plots
fig_e1 = plotter.generic_plot(
    t, 
    *[r['analysis']['e1'] for r in selected_sim_results],
    xlabel="$t \\ [\\text{s}]$", 
    ylabel="$e_1 \\ [\\text{m}]$", 
    title="Constraint 1 — Surface contact: $e_1 = S(p_x^{\\text{task}}, p_y^{\\text{task}}) - p_z^{\\text{task}}$", 
    #labels=[r['name'] for r in selected_sim_results]
    labels = [
    r"$\mu = 10^{-4}$",
    r"$\mu = 10^{-3}$",
    r"$\mu = 10^{-2}$",
    r"$\mu = 10^{-1}$",
    r"$\mu = 10^{0}$"
]
)

fig_e2 = plotter.generic_plot(
    t, 
    *[r['analysis']['e2'] for r in selected_sim_results],
    xlabel="$t \\ [\\text{s}]$", 
    ylabel="$e_2$", 
    title="Constraint 2 — Normal alignment: $e_2 = \\frac{\\mathbf{n}^{\\mathsf{T}}}{\\lVert \\mathbf{n} \\rVert} \\mathbf{r}_z - 1$", 
    #labels=[r['name'] for r in selected_sim_results]
    labels = [
    r"$\mu = 10^{-4}$",
    r"$\mu = 10^{-3}$",
    r"$\mu = 10^{-2}$",
    r"$\mu = 10^{-1}$",
    r"$\mu = 10^{0}$"
]
)

fig_e3 = plotter.generic_plot(
    t, 
    *[r['analysis']['e3'] for r in selected_sim_results],
    xlabel="$t \\ [\\text{s}]$", 
    ylabel="$e_3$", 
    title="Constraint 3 — Lateral axis orthogonality: $e_3 = \\begin{bmatrix} 1 & 0 & 0 \\end{bmatrix} \\mathbf{r}_y - 0$", 
    #labels=[r['name'] for r in selected_sim_results]
    labels = [
    r"$\mu = 10^{-4}$",
    r"$\mu = 10^{-3}$",
    r"$\mu = 10^{-2}$",
    r"$\mu = 10^{-1}$",
    r"$\mu = 10^{0}$"
]
)

fig_e4 = plotter.generic_plot(
    t, 
    *[r['analysis']['e4'] for r in selected_sim_results],
    xlabel="$t \\ [\\text{s}]$", 
    ylabel="$e_4 \\ [\\text{m}]$", 
    title="Constraint 4 — Fixed x position: $e_4 = p_x^{\\text{task}} - p_{x,\\mathrm{ref}}$", 
    #labels=[r['name'] for r in selected_sim_results]
    labels = [
    r"$\mu = 10^{-4}$",
    r"$\mu = 10^{-3}$",
    r"$\mu = 10^{-2}$",
    r"$\mu = 10^{-1}$",
    r"$\mu = 10^{0}$"
]
)

fig_e5 = plotter.generic_plot(
    t, 
    *[r['analysis']['e5'] for r in selected_sim_results],
    xlabel="$t \\ [\\text{s}]$", 
    ylabel="$e_5 \\ [\\text{m/s}]$", 
    title="Constraint 5 — Tangential velocity: $e_5 = v^{\\text{task}}_{y} - v_{y,\\mathrm{ref}}$", 
    #labels=[r['name'] for r in selected_sim_results]
    labels = [
    r"$\mu = 10^{-4}$",
    r"$\mu = 10^{-3}$",
    r"$\mu = 10^{-2}$",
    r"$\mu = 10^{-1}$",
    r"$\mu = 10^{0}$"
]
)

# x = [q1..q6, qdot1..qdot6]
fig_qdot1 = plotter.generic_plot(
    t,
    *[r['data']['qdot'][0, :] for r in selected_sim_results],
    xlabel=r"$t\ [\mathrm{s}]$",
    ylabel=r"$\dot q_1\ [\mathrm{rad/s}]$",
    title="Joint velocity - Joint 1",
    labels = [
    r"$\mu = 10^{-4}$",
    r"$\mu = 10^{-3}$",
    r"$\mu = 10^{-2}$",
    r"$\mu = 10^{-1}$",
    r"$\mu = 10^{0}$"
]
)
fig_u1 = plotter.generic_plot(
    t,
    *[r['data']['qdot'][0, :] for r in selected_sim_results],
    xlabel=r"$t\ [\mathrm{s}]$",
    ylabel=r"$\dot q_1\ [\mathrm{rad/s}]$",
    title="Joint velocity - Joint 1",
    labels = [
    r"$\mu = 10^{-4}$",
    r"$\mu = 10^{-3}$",
    r"$\mu = 10^{-2}$",
    r"$\mu = 10^{-1}$",
    r"$\mu = 10^{0}$"
]
)



# Timing Performance
fig_mpc_time = plotter.generic_plot(
    t, 
    *[r['analysis']['mpc_time'] for r in selected_sim_results],
    upper_bound=selected_sim_results[0]['simulator'].dt,
    xlabel="$t \\ [\\text{s}]$", 
    ylabel="$\\text{MPC Time} \\ [\\text{s}]$", 
    title=f"MPC Computation Time (dt = {selected_sim_results[0]['simulator'].dt}s)", 
    labels=[r['name'] for r in selected_sim_results]
)
fig_integration_time = plotter.generic_plot(
    t, 
    *[r['analysis']['integration_time'] for r in selected_sim_results],
    upper_bound=selected_sim_results[0]['simulator'].dt,
    xlabel="$t \\ [\\text{s}]$", 
    ylabel="$\\text{Integration Time} \\ [\\text{s}]$", 
    title=f"Integration Computation Time (dt = {selected_sim_results[0]['simulator'].dt}s)", 
    labels=[r['name'] for r in selected_sim_results]
)

rmse_data = {
    "$e_1$ (surface)": [r['summary']['rmse_e1'] for r in sim_results],
    "$e_2$ (normal alignment)": [r['summary']['rmse_e2'] for r in sim_results],
    "$e_3$ (lateral alignment)": [r['summary']['rmse_e3'] for r in sim_results],
    "$e_4$ (x-pos)": [r['summary']['rmse_e4'] for r in sim_results],
    "$e_5$ (y-vel)": [r['summary']['rmse_e5'] for r in sim_results],
}
fig_rmse = plotter.grouped_bar_plot(
    data=rmse_data,
    group_labels=[r['name'] for r in sim_results],
    xlabel="Simulation",
    ylabel="RMSE",
    title="Constraint RMSE Comparison"
)
fig_mpc_time = plotter.generic_plot(
    t, 
    *[r['analysis']['mpc_time'] for r in selected_sim_results],
    upper_bound=selected_sim_results[0]['simulator'].dt,
    xlabel="$t \\ [\\text{s}]$", 
    ylabel="$\\text{MPC Time} \\ [\\text{s}]$", 
    title=f"MPC Computation Time (dt = {selected_sim_results[0]['simulator'].dt}s)", 
    labels=[r['name'] for r in selected_sim_results]
)



for fig in [fig_e1, fig_e2, fig_e3, fig_e4, fig_e5]:
    plotter.show(fig)


'''
display(fig_e1, fig_e2, fig_e3, fig_e4, fig_e5, fig_u1)
display(fig_rmse, fig_total_time, fig_mpc_time)

display(fig_box)
'''


/root/miniforge3/lib/python3.12/site-packages/plotly/offline/offline.py:152: UserWarning:


Unrecognized config options supplied: ['mathjax']

/root/miniforge3/lib/python3.12/site-packages/plotly/offline/offline.py:152: UserWarning:


Unrecognized config options supplied: ['mathjax']

/root/miniforge3/lib/python3.12/site-packages/plotly/offline/offline.py:152: UserWarning:


Unrecognized config options supplied: ['mathjax']

/root/miniforge3/lib/python3.12/site-packages/plotly/offline/offline.py:152: UserWarning:


Unrecognized config options supplied: ['mathjax']

/root/miniforge3/lib/python3.12/site-packages/plotly/offline/offline.py:152: UserWarning:


Unrecognized config options supplied: ['mathjax']



'\ndisplay(fig_e1, fig_e2, fig_e3, fig_e4, fig_e5, fig_u1)\ndisplay(fig_rmse, fig_total_time, fig_mpc_time)\n\ndisplay(fig_box)\n'

In [7]:
from plotter import Plotter
plotter = Plotter()
fig_qdot1 = plotter.generic_plot(
    t,
    *[r['data']['qdot'][0, :] for r in selected_sim_results],
    xlabel=r"$t\ [\mathrm{s}]$",
    ylabel=r"$\dot q_1\ [\mathrm{rad/s}]$",
    title="Joint velocity - Joint 1",
    labels = [
    r"$\mu = 10^{-4}$",
    r"$\mu = 10^{-3}$",
    r"$\mu = 10^{-2}$",
    r"$\mu = 10^{-1}$",
    r"$\mu = 10^{0}$"
]
)
fig_qdot2 = plotter.generic_plot(
    t,
    *[r['data']['qdot'][1, :] for r in selected_sim_results],
    xlabel=r"$t\ [\mathrm{s}]$",
    ylabel=r"$\dot q_2\ [\mathrm{rad/s}]$",
    title="Joint velocity - Joint 2",
    labels = [
    r"$\mu = 10^{-4}$",
    r"$\mu = 10^{-3}$",
    r"$\mu = 10^{-2}$",
    r"$\mu = 10^{-1}$",
    r"$\mu = 10^{0}$"
]
)
fig_qdot3 = plotter.generic_plot(
    t,
    *[r['data']['qdot'][2, :] for r in selected_sim_results],
    xlabel=r"$t\ [\mathrm{s}]$",
    ylabel=r"$\dot q_3\ [\mathrm{rad/s}]$",
    title="Joint velocity - Joint 3",
    labels = [
    r"$\mu = 10^{-4}$",
    r"$\mu = 10^{-3}$",
    r"$\mu = 10^{-2}$",
    r"$\mu = 10^{-1}$",
    r"$\mu = 10^{0}$"
]
)
fig_qdot4 = plotter.generic_plot(
    t,
    *[r['data']['qdot'][3, :] for r in selected_sim_results],
    xlabel=r"$t\ [\mathrm{s}]$",
    ylabel=r"$\dot q_4\ [\mathrm{rad/s}]$",
    title="Joint velocity - Joint 4",
    labels = [
    r"$\mu = 10^{-4}$",
    r"$\mu = 10^{-3}$",
    r"$\mu = 10^{-2}$",
    r"$\mu = 10^{-1}$",
    r"$\mu = 10^{0}$"
]
)
fig_qdot5 = plotter.generic_plot(
    t,
    *[r['data']['qdot'][4, :] for r in selected_sim_results],
    xlabel=r"$t\ [\mathrm{s}]$",
    ylabel=r"$\dot q_5\ [\mathrm{rad/s}]$",
    title="Joint velocity - Joint 5",
    labels = [
    r"$\mu = 10^{-4}$",
    r"$\mu = 10^{-3}$",
    r"$\mu = 10^{-2}$",
    r"$\mu = 10^{-1}$",
    r"$\mu = 10^{0}$"
]
)
fig_qdot6 = plotter.generic_plot(
    t,
    *[r['data']['qdot'][5, :] for r in selected_sim_results],
    xlabel=r"$t\ [\mathrm{s}]$",
    ylabel=r"$\dot q_6\ [\mathrm{rad/s}]$",
    title="Joint velocity - Joint 6",
    labels = [
    r"$\mu = 10^{-4}$",
    r"$\mu = 10^{-3}$",
    r"$\mu = 10^{-2}$",
    r"$\mu = 10^{-1}$",
    r"$\mu = 10^{0}$"
]
)

fig_u1 = plotter.generic_plot(
    t,
    *[r['data']['u'][0, :] for r in selected_sim_results],
    xlabel=r"$t\ [\mathrm{s}]$",
    ylabel=r"$\dot q_{1_ref}\ [\mathrm{rad/s}]$",
    title="Joint refrence velocity - Joint 1",
    labels = [
    r"$\mu = 10^{-4}$",
    r"$\mu = 10^{-3}$",
    r"$\mu = 10^{-2}$",
    r"$\mu = 10^{-1}$",
    r"$\mu = 10^{0}$"
]
)
for fig in [fig_qdot1, fig_qdot2, fig_qdot3, fig_qdot4, fig_qdot5, fig_qdot6, fig_u1]:
    plotter.show(fig)

/root/miniforge3/lib/python3.12/site-packages/plotly/offline/offline.py:152: UserWarning:


Unrecognized config options supplied: ['mathjax']

/root/miniforge3/lib/python3.12/site-packages/plotly/offline/offline.py:152: UserWarning:


Unrecognized config options supplied: ['mathjax']

/root/miniforge3/lib/python3.12/site-packages/plotly/offline/offline.py:152: UserWarning:


Unrecognized config options supplied: ['mathjax']

/root/miniforge3/lib/python3.12/site-packages/plotly/offline/offline.py:152: UserWarning:


Unrecognized config options supplied: ['mathjax']

/root/miniforge3/lib/python3.12/site-packages/plotly/offline/offline.py:152: UserWarning:


Unrecognized config options supplied: ['mathjax']

/root/miniforge3/lib/python3.12/site-packages/plotly/offline/offline.py:152: UserWarning:


Unrecognized config options supplied: ['mathjax']

/root/miniforge3/lib/python3.12/site-packages/plotly/offline/offline.py:152: UserWarning:


Unrecognized config options supplied: ['mathjax']


In [8]:
import plotly.io as pio
pio.renderers.default = "browser"


In [28]:
import importlib, plotter
importlib.reload(plotter)
plotter = plotter.Plotter()


/root/robotic-mpc/plotter.py:95: SyntaxWarning:

invalid escape sequence '\d'



In [31]:
#fig_q = plotter.joints(t, *[r['data']['q'] for r in selected_sim_results], labels=[r['name'] for r in selected_sim_results], lower_bounds=JOINT_POSITION_LIMITS, upper_bounds=JOINT_POSITION_LIMITS)
fig_qdot = plotter.joints(t[:1200], *[r['data']['qdot'] for r in selected_sim_results], labels = [
    r"$\mu = 10^{-4}$",
    r"$\mu = 10^{-3}$",
    r"$\mu = 10^{-2}$",
    r"$\mu = 10^{-1}$",
    r"$\mu = 10^{0}$"
], lower_bounds=JOINT_VELOCITY_LIMITS_LOW, upper_bounds=JOINT_VELOCITY_LIMITS_UP)
plotter.show(fig_qdot)

/root/miniforge3/lib/python3.12/site-packages/plotly/offline/offline.py:152: UserWarning:


Unrecognized config options supplied: ['mathjax']



In [17]:

res = sim_results[0]['analysis']
fig_res_components = plotter.generic_plot(
    t,
    res['res_stat'],
    res['res_eq'],
    res['res_ineq'],
    res['res_comp'],
    ylog=True,
    xlabel="$t$ [s]",
    ylabel="Residual",
    title=f"Residual Components - {sim_results[0]['name']}",
    labels=["Stationarity", "Equality", "Inequality", "Complementarity"]
)

# SQP Iterations
fig_sqp = plotter.generic_plot(
    t,
    *[r['analysis']['sqp_iterations'] for r in selected_sim_results],
    xlabel="$t$ [s]",
    ylabel="SQP Iterations",
    title="SQP Iterations per MPC Solve",
    labels=[r['name'] for r in selected_sim_results]
)

# Cost history - Should be final cost at each iteration.
fig_cost = plotter.generic_plot(
    i ,
    *[r['analysis']['cost_history'] for r in selected_sim_results],
    ylog=True,
    xlabel="$i$ [s]",
    ylabel="Cost",
    title="Final Cost at each simulation step",
    labels=[r['name'] for r in selected_sim_results]
)



display(fig_res_components, fig_sqp, fig_cost, fig_mpc_time)

In [18]:
plotter.gen_html_report(
    task_figs=[fig_e1, fig_e2, fig_e3, fig_e4, fig_e5, fig_u1, fig_q, fig_qdot, fig_q, fig_qdot],
    solver_figs=[fig_res_components, fig_sqp, fig_cost, fig_mpc_time],
    video_folder=None,
    title="Surface Tracking Analysis",
    filename="surface_tracking_analysis.html"
)

'/root/Reporting/HTML/surface_tracking_analysis.html'